<a href="https://colab.research.google.com/github/billsioros/thesis/blob/master/Nanorough_surface_Super_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

## Pip Modules

In [2]:
!pip install torch

## RNG Initialization

We are going to be seed the random number generator engines, so that our results are reproducible accross different set ups

In [3]:
import torch
import numpy as np
import random
import os

SEED = 1234

if SEED is not None:
  np.random.seed(SEED)
  random.seed(SEED)
  torch.manual_seed(SEED)
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  os.environ['PYTHONHASHSEED'] = str(SEED)

## Determining the Current Working Directory

In [4]:
from pathlib import Path

BASE_DIR = Path.cwd()
BASE_DIR.mkdir(parents=True, exist_ok=True)

# Dataset Loading and Preprocessing

In [ ]:
from torch.utils.data.dataset import  Dataset

class NanoroughDataset(Dataset):
  def __init__(self):
    pass

  def __len__(self):
    pass
  
  def __getitem__(self):
    pass

# Training and Testing Utilities

In [ ]:
from time import time
from functools import wraps

def benchmark(method=None, *, debug=False):
  def _(method):
    @wraps(method)
    def wrapper(*args, **kwargs):
      beg = time()
      rv = method(*args, **kwargs)
      end = time()

      if debug is True:
        args_str = ', '.join(str(x) for x in args)
        kwargs_str = ', '.join("{}={}".format(x, y) for x, y in kwargs.items())

        print("%s(%s, %s) returned %s after %7.3f seconds" % (method.__name__, args_str, kwargs_str, str(rv), (end - beg)))
      else:
        print("%s returned after %7.3f seconds" % (method.__name__, (end - beg)))

      return rv

    return wrapper
  
  return _ if method is None else _(method)

In [ ]:
@benchmark
def train_epoch(model, bucket_iterator, optimizer, criterion, log_every_n=None):
  model.train()

  train_loss = 0
  for train_iteration, batch in enumerate(bucket_iterator):
    if log_every_n is not None and not train_iteration % log_every_n:
      print(f"Training Iteration #{train_iteration:04d}")

    optimizer.zero_grad()

    y_pred = model(X_batch).squeeze()
    
    loss = criterion(y_pred, y_batch)
    loss.backward()

    optimizer.step()
    
    train_loss += loss.item() / len(bucket_iterator)

  return train_loss

In [ ]:
from torch import no_grad

@benchmark
def test_epoch(model, bucket_iterator, criterion, log_every_n=None):
  model.eval()

  test_loss =  0
  with no_grad():
    for test_iteration, batch in enumerate(bucket_iterator):
      X_batch, y_batch = batch.tweet, batch.sentiment
      if log_every_n is not None and not test_iteration % log_every_n:
        print(f"Testing Iteration #{test_iteration:04d}")

      y_pred = model(X_batch).squeeze()

      loss = criterion(y_pred, y_batch)
      
      test_loss += loss.item() / len(bucket_iterator)
    
  return test_loss

# A naive-approach

In [1]:
from torch import nn

In [9]:
class PerceptronNetwork(nn.Module):
  def __init__(self, in_features, out_features):
    super().__init__()

    self.linear = nn.Linear(in_features, out_features)
    self.activation = nn.ReLU()
  
  def forward(self, batch):
    return self.activation(self.linear(batch))

In [ ]:
CHECKPOINT_DIR, N_EPOCHS, LEARNING_RATE, WEIGHT_DECAY = None, 10, 0.001, 0

In [11]:
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import MSELoss

device = "cpu"
if torch.cuda.is_available():
  device = "cuda:0"
  if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

device = torch.device(device)

model = model.to(device)

criterion = MSELoss.to(device)

optimizer = Adam(filter(lambda x: x.requires_grad, PerceptronNetwork(2, 2).parameters()), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = ReduceLROnPlateau(optimizer=optimizer)

# train_iter, test_iter = data.BucketIterator.splits(
#   (train_data, test_data),
#   sort_key=lambda x: len(x.tweet),
#   shuffle=True,
#   batch_size=256,
#   device=device
# )

train_losses, test_losses = [], []
for epoch in range(n_epochs):
  train_loss = train_epoch(model, train_iter, optimizer, criterion, log_every_n)

  test_loss = test_epoch(model, test_iter, criterion, log_every_n)
  
  if CHECKPOINT_DIR is not None and (not test_losses or test_loss < min(test_losses)):
    torch.save(model.state_dict(), CHECKPOINT_DIR / f'{epoch:03d}.mt')

  print("Epoch: %02d, Train Loss: %7.3f, Test Loss: %7.3f" % (epoch, train_loss, test_loss))

  if scheduler is not None:
    scheduler.step(test_loss)

NameError: ignored